# 第3部　Pythonによるデータ分析｜Pythonで学ぶ統計学入門

## 7章　推定

### 分析の準備

In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# グラフを描画するライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# 表示桁数の指定
%precision 3
# グラフをjupyter Notebook内に表示させるための指定
%matplotlib inline

In [2]:
# データの読み込み
fish = pd.read_csv("3-7-1-fish_length.csv")["length"]
fish

0    4.352982
1    3.735304
2    5.944617
3    3.798326
4    4.087688
5    5.265985
6    3.272614
7    3.526691
8    4.150083
9    3.736104
Name: length, dtype: float64

### 実装：点推定

In [3]:
# 母平均の点推定
mu = sp.mean(fish)
mu

4.187

In [4]:
# 母分散の点推定
sigma_2 = sp.var(fish, ddof = 1)
sigma_2

0.680

### 実装：区間推定

In [6]:
# 自由度
df = len(fish) - 1
df

9

In [7]:
# 標準誤差
se = sigma / sp.sqrt(len(fish))
se

0.261

In [8]:
# 区間推定
interval = stats.t.interval(
    alpha = 0.95, df = df, loc = mu, scale = se)
interval

(3.597, 4.777)

### 補足：信頼区間の求め方の詳細

In [9]:
# 97.5%点
t_975 = stats.t.ppf(q = 0.975, df = df)
t_975

2.262

In [10]:
# 下側信頼限界
lower = mu - t_975 * se
lower

3.597

In [11]:
# 上側信頼限界
upper = mu + t_975 * se
upper

4.777

### 信頼区間の幅を決める要素

In [12]:
# 標本標準偏差が大きいと、信頼区間は広くなる
se2 = (sigma*10) / sp.sqrt(len(fish))
stats.t.interval(
    alpha = 0.95, df = df, loc = mu, scale = se2)

(-1.713, 10.087)

In [13]:
# サンプルサイズが大きいと、信頼区間は狭くなる
df2 = (len(fish)*10) - 1
se3 = sigma / sp.sqrt(len(fish)*10)
stats.t.interval(
    alpha = 0.95, df = df2, loc = mu, scale = se3)

(4.023, 4.351)

In [14]:
# 99%信頼区間
stats.t.interval(
    alpha = 0.99, df = df, loc = mu, scale = se)

(3.339, 5.035)

### 信頼区間の解釈

In [19]:
# 信頼区間が母平均(4)を含んでいればTrue
be_included_array = np.zeros(20000, dtype = "bool")
be_included_array

array([False, False, False, ..., False, False, False], dtype=bool)

In [20]:
# 「データを10個選んで95%信頼区間を求める」試行を20000回繰り返す
# 信頼区間が母平均(4)を含んでいればTrue
np.random.seed(1)
norm_dist = stats.norm(loc = 4, scale = 0.8)
for i in range(0, 20000):
    sample = norm_dist.rvs(size = 10)
    df = len(sample) - 1
    mu = sp.mean(sample)
    std = sp.std(sample, ddof = 1)
    se = std / sp.sqrt(len(sample))
    interval = stats.t.interval(0.95, df, mu, se)
    if(interval[0] <= 4 and interval[1] >= 4):
        be_included_array[i] = True

In [21]:
sum(be_included_array) / len(be_included_array)

0.948